In [1]:
#ucitavamo biblioteke
import tkinter as tk
from tkinter import *
from PIL import Image
from PIL import ImageTk
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
import os
from tensorflow.keras.optimizers import Adam
import threading

In [2]:

# Pravimo istu arhitekturu kao i na treniranje modela, s tim da ovdje ne treniramo model, vec dodajemo one sacuvane weights
face_model = Sequential()
face_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
face_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
face_model.add(MaxPooling2D(pool_size=(2, 2)))
face_model.add(Dropout(0.25))
face_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
face_model.add(MaxPooling2D(pool_size=(2, 2)))
face_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
face_model.add(MaxPooling2D(pool_size=(2, 2)))
face_model.add(Dropout(0.25))
face_model.add(Flatten())
face_model.add(Dense(1024, activation='relu'))
face_model.add(Dropout(0.5))
face_model.add(Dense(7, activation='softmax'))

# Load the saved weights
face_model.load_weights('recognition_model.h5')

# Ovo ne znam sta tacno radi, vidio sam da je upisano na nekim radovima :) 
cv2.ocl.setUseOpenCL(False)

# Nazivi emocija 

emotion_dict = {0: "   Angry   ", 1: "Disgusted", 2: "  Fearful  ", 3: "   Happy   ", 4: "  Neutral  ", 5: "    Sad   ", 6: "Surprised"}
#ucitavamo slike emojija, ovi sa brojem 1 kao lice na mene(Ognjena), a sa brojem 2 na Stasu
emoji_dist = {0:"emojis/Angry1.png", 1:"emojis/Disgusted1.png", 2:"emojis/Fearful1.png", 3:"emojis/Happy1.png", 4:"emojis/Neutral1.png", 5:"emojis1/Sad1.png", 6:"emojis/Surprised1.png"}
#emoji_dist = {0:"emojis/Angry2.png", 1:"emojis/Disgusted2.png", 2:"emojis/Fearful2.png", 3:"emojis/Happy2.png", 4:"emojis/Neutral2.png", 5:"emojis/Sad2.png", 6:"emojis/Surprised2.png"}


####################################
# Procitajte prvo donji box jer je to glavni dio gui-a, ovo su funkcije koje se u njemu koriste 


# Global variables
global last_frame1                                    
last_frame1 = np.zeros((480, 640, 3), dtype=np.uint8) #ovo je prvi frame,koji je prazan, da bi mogao da se pokrene video
global cap1
show_text=[0]
global frame_number

# Funkcija koja pokrece video ili video kameru u zavisnosti od toga sta zelimo
def show_subject():
    global cap1
    cap1 = cv2.VideoCapture(r'Klipovi\Naziv_videa.mp4') #ovdje ubacite video, nadjite kad ucitate koji je tacan path,
    #ovo r ispred '' valjda sluzi da \ procita bukvalno a ne da nesto radi
    
    #ukoliko zelimo da ucitavamo video sa video kamere
    #cap1 = cv2.VideoCapture(0)
    
    #ukoliko ne ucita ni kameru ni video izbacuje gresku 
    if not cap1.isOpened():
        print("cant open the camera")
    
    #ovaj dio koda valjda radi da ugasi gui kad se zavrsi snimak, al nisam siguran da meni radi :), moze i bez ovoga 
    global frame_number
    length = int(cap1.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_number += 1
    if frame_number >= length:
        exit()
    
    #ovo je dio koji prepoznaje lice
    cap1.set(1, frame_number)    
    flag1, frame1 = cap1.read() #ucitava frame
    frame1 = cv2.resize(frame1,(400,500)) #### ovaj broj piksela korigujte ako ocete video kameru da otvorite, meni je za kameru 600,500 odgovara
    bound_box = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') #ovaj fajl sluzi za to prepoznavanje
    gray_frame = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY) #ovo pretvara sliku u crno belu, jer smo program trenirali na crno belim slikama
    n_faces = bound_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5) #ovo je valjda nesto sto u sustini radi da ne prepoznaje isto lice vise puta

    #ovdje crtamo pravougaonik oko lica
    #zatim secemo da slika bude 48,48 kao i podaci
    #predvidjamo emociju i ispisujemo iznad glave
    #i cuvamo redni broj emocije da bismo posle nacrtali odgovarajuci emoji
    for (x, y, w, h) in n_faces:
        cv2.rectangle(frame1, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_frame = gray_frame[y:y + h, x:x + w]
        crop_img = np.expand_dims(np.expand_dims(cv2.resize(roi_frame, (48, 48)), -1), 0)
        prediction = face_model.predict(crop_img)
        maxindex = int(np.argmax(prediction))
        cv2.putText(frame1, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        show_text[0]=maxindex
    
    #flag nam je isto sto i frame, ovdje detektuje da li postoji, i ispisuje gresku ako ne
    if flag1 is None:
        print ("Error!")
    
    #ovo je dio koda koji prebacuje na sledeci frejm, ne znam tacno zasto je neophodan koji korak, mrzi me da analiziram
    
    elif flag1:
        global last_frame1
        last_frame1 = frame1.copy()
        pic = cv2.cvtColor(last_frame1, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(pic)
        imgtk = ImageTk.PhotoImage(image=img)
        lmain.imgtk = imgtk
        lmain.configure(image=imgtk)
        root.update()
        lmain.after(1, show_subject)
        
    #klikom na slovo q se prekida program, mada meni nesto ne radi kako treba nzm
    if cv2.waitKey(1) & 0xFF == ord('q'):
        exit()

#Funckija za prikazivanje emodzija, tu nema nista bitno u kodu, samo estetika 
def show_avatar():
    frame2=cv2.imread(emoji_dist[show_text[0]])
    pic2=cv2.cvtColor(frame2,cv2.COLOR_BGR2RGB)
    img2=Image.fromarray(pic2)
    imgtk2=ImageTk.PhotoImage(image=img2)
    lmain2.imgtk2=imgtk2
    lmain2.configure(image=imgtk2)
    lmain2.after(1, show_avatar)



In [3]:
#ovo je glavni dio koda, 
if __name__ == '__main__':
    frame_number = 0
    root=tk.Tk() #ovo je taj gui
    lmain = tk.Label(master=root,padx=50,bd=10) #ovo je box u kome se nalazi snimak, ili kamera
    lmain2 = tk.Label(master=root,bd=10, bg = 'black') #ovo je box u kome se nalazi emoji
    #ovo je samo estetika
    lmain.pack(side=LEFT)
    lmain.place(x=50,y=120)
    lmain2.pack(side=RIGHT)
    lmain2.place(x=800,y=70)
    root.title("Emojify")
    root.geometry("1400x900+100+10")
    root['bg']='black'
    #dugme za prekidanje programa, kada ocete da ugasite klikajte ovo, a ne x na prozoru, jer zablokira kernel
    exitbutton = Button(root, text='QUIT',fg="red",command=root.destroy,font=('arial',25,'bold')).pack(side = BOTTOM)
    #prikazi video
    threading.Thread(target=show_subject).start()
    #prikazi emoji
    threading.Thread(target=show_avatar).start()
    #ponovi
    root.mainloop()

1/1 [==============================] - 0s 16ms/step
